# Plan

- Goal: Build a strong, GPU-accelerated image classification pipeline for iWildCam 2019 with reliable CV and fast iterations. Target: ≥ bronze (macro-F1 ≥ 0.114), aim ≥ silver.
- Validation:
  - Stratified KFold on category_id (5 folds). If metadata like site/sequence is available, consider GroupKFold by location/sequence; otherwise stick to StratifiedKFold for speed initially.
- Baseline model:
  - Pretrained timm backbone (e.g., convnext_tiny or efficientnet_b0) fine-tuned with mixed precision.
  - Input size 224, augmentations (RandomResizedCrop, HFlip, ColorJitter, AutoAugment).
  - Class-balanced sampler or weighted loss (CrossEntropy with class weights or focal).
  - Early stopping, cosine schedule, warmup.
- Iteration steps:
  1) Environment + GPU check
  2) Data load & sanity checks; unzip images (to /mnt for I/O speed), verify paths
  3) Fast smoke run on small subset (e.g., 5k images, 1 fold) to validate pipeline
  4) Full training with 5-fold CV; save OOF and test logits
  5) Error analysis; try alternative backbones (beit_base_224, eva02, nfnet), resolutions 256/320; tune class weights
  6) Blend top-2 models by logits

- Deliverables:
  - Deterministic folds saved
  - submission.csv
  - Logged training progress per epoch/fold

Request expert review after: plan (now), data EDA, baseline smoke OOF, first full CV, and any poor score.

In [2]:
# Environment + Data EDA + Prepare fast storage
import os, sys, shutil, time, json, math, subprocess, zipfile, io
import pandas as pd
from pathlib import Path

print('=== GPU CHECK (nvidia-smi) ===', flush=True)
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    print('nvidia-smi failed:', e)

print('=== Python & Disk Info ===')
print(sys.version)
print('CWD:', os.getcwd())
print('Free space on / and CWD (GB):', shutil.disk_usage('/').free/1e9, shutil.disk_usage('.').free/1e9)

DATA_DIR = Path('.')
# Use a writable local fast directory instead of /mnt (permission denied)
FAST_DIR = Path('./fast_storage/iwildcam')
FAST_DIR.mkdir(parents=True, exist_ok=True)

train_csv = DATA_DIR/'train.csv'
test_csv = DATA_DIR/'test.csv'
train_zip = DATA_DIR/'train_images.zip'
test_zip = DATA_DIR/'test_images.zip'

print('=== Load CSVs ===')
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
print('train_df shape:', train_df.shape)
print('test_df shape:', test_df.shape)
print('Columns:', list(train_df.columns))
print('Head:\n', train_df.head(3))

# Identify potential grouping columns to avoid leakage
candidate_groups = [c for c in ['sequence_id','location','site','seq_id','sequence','location_id'] if c in train_df.columns]
print('Candidate group columns:', candidate_groups)
for c in candidate_groups:
    vc = train_df[c].value_counts().head(5)
    print(f'Top groups for {c}:\n{vc}')

# Target distribution
target_col = 'category_id'
if target_col in train_df.columns:
    n_classes = train_df[target_col].nunique()
    print('Num classes:', n_classes)
    print('Target sample:\n', train_df[target_col].value_counts().head(10))

# Plan image extraction to fast disk if not already extracted
fast_train_dir = FAST_DIR/'train_images'
fast_test_dir = FAST_DIR/'test_images'

def unzip_if_needed(zip_path: Path, out_dir: Path, label: str):
    if out_dir.exists() and any(out_dir.iterdir()):
        print(f'{label}: already extracted at {out_dir}')
        return
    out_dir.mkdir(parents=True, exist_ok=True)
    t0 = time.time()
    print(f'Extracting {zip_path} -> {out_dir} ...', flush=True)
    # Use system unzip for speed if available
    if shutil.which('unzip') is not None:
        cmd = f"unzip -qq -o '{zip_path}' -d '{out_dir}'"
        print('Running:', cmd)
        rc = subprocess.call(['bash','-lc', cmd])
        if rc != 0:
            print('unzip failed with code', rc, '; falling back to python zipfile')
        else:
            print(f'{label}: unzip completed in {time.time()-t0:.1f}s')
            return
    # Fallback: python zipfile (slower)
    with zipfile.ZipFile(zip_path) as zf:
        members = zf.namelist()
        n = len(members)
        for i, m in enumerate(members, 1):
            zf.extract(m, out_dir)
            if i % 5000 == 0:
                print(f'{label}: extracted {i}/{n}', flush=True)
    print(f'{label}: extraction completed in {time.time()-t0:.1f}s')

print('=== Ensure images on local fast storage (./fast_storage) ===')
if train_zip.exists():
    unzip_if_needed(train_zip, fast_train_dir, 'train')
else:
    print('Missing train_images.zip')
if test_zip.exists():
    unzip_if_needed(test_zip, fast_test_dir, 'test')
else:
    print('Missing test_images.zip')

# Sanity: sample a few image paths from train_df
img_col_candidates = [c for c in ['file_name','filename','image_id','id','image_path'] if c in train_df.columns]
print('Image id/path columns candidates:', img_col_candidates)
if img_col_candidates:
    img_col = img_col_candidates[0]
    print('Using image column:', img_col)
    # Try to construct absolute paths under fast dir
    sample_rows = train_df.head(5)
    for _, r in sample_rows.iterrows():
        fn = str(r[img_col])
        # Many competitions store just filename; adapt if nested
        p1 = fast_train_dir/fn
        # If not exists, try basename
        if not p1.exists():
            p1 = fast_train_dir/Path(fn).name
        print('Sample image path candidate:', p1, 'exists:', p1.exists())
else:
    print('Could not identify image filename column; will inspect in later step.')

print('=== EDA step complete ===')

# Next steps (for operator log):
# - Decide grouping key for CV based on candidate_groups (prefer sequence_id, else location).
# - Build folds and persist to disk.
# - Implement dataloaders and baseline model training script.

=== GPU CHECK (nvidia-smi) ===


Wed Sep 24 19:30:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train_df shape: (179422, 11)
test_df shape: (16877, 10)
Columns: ['category_id', 'date_captured', 'file_name', 'frame_num', 'id', 'location', 'rights_holder', 'seq_id', 'seq_num_frames', 'width', 'height']
Head:
    category_id        date_captured                                 file_name  \
0           19  2012-03-17 03:48:44  588a679f-23d2-11e8-a6a3-ec086b02610b.jpg   
1            0  2014-05-11 11:56:46  59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg   
2            0  2013-10-06 02:00:00  5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg   

   frame_num                                    id  location  rights_holder  \
0          2  588a679f-23d2-11e8-a6a3-ec086b02610b       115   Justin Brown   
1          1  59279ce3-23d2-11e8-a6a3-ec086b02610b        96  Erin Boydston   
2          1  5a2af4ab-23d2-11e8-a6a3-ec086b02610b        57  Erin Boydston   

                                 seq_id  seq_num_frames  width  height  
0  6f12067d-5567-11e8-b3c0-dca9047ef277               3   1024     747  

train: extracted 5000/179224


train: extracted 10000/179224


...

train: extraction completed in 93.2s
Extracting test_images.zip -> fast_storage/iwildcam/test_images ...


test: extracted 5000/16862


test: extracted 10000/16862


test: extracted 15000/16862


test: extraction completed in 7.7s
Image id/path columns candidates: ['file_name', 'id']
Using image column: file_name
Sample image path candidate: fast_storage/iwildcam/train_images/588a679f-23d2-11e8-a6a3-ec086b02610b.jpg exists: True
Sample image path candidate: fast_storage/iwildcam/train_images/59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg exists: True
Sample image path candidate: fast_storage/iwildcam/train_images/5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg exists: True
Sample image path candidate: fast_storage/iwildcam/train_images/593d68d7-23d2-11e8-a6a3-ec086b02610b.jpg exists: True
Sample image path candidate: fast_storage/iwildcam/train_images/58782b45-23d2-11e8-a6a3-ec086b02610b.jpg exists: True
=== EDA step complete ===


In [3]:
# Build grouped CV folds, label maps, and class weights
import pandas as pd, numpy as np, json
from pathlib import Path
from collections import Counter
from sklearn.model_selection import GroupKFold
try:
    from sklearn.model_selection import StratifiedGroupKFold
    HAS_SGK = True
except Exception:
    HAS_SGK = False

DATA_DIR = Path('.')
FAST_DIR = Path('./fast_storage/iwildcam')
train_df = pd.read_csv(DATA_DIR/'train.csv')

# Determine groups and target
target_col = 'category_id'
group_col = 'seq_id' if 'seq_id' in train_df.columns else ('location' if 'location' in train_df.columns else None)
assert group_col is not None, 'No grouping column found (seq_id/location missing)'
y = train_df[target_col].values
groups = train_df[group_col].values

n_splits = 5
rng_seed = 42

folds = np.full(len(train_df), -1, dtype=int)
if HAS_SGK:
    sgk = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=rng_seed)
    for fi, (_, val_idx) in enumerate(sgk.split(train_df, y, groups=groups)):
        folds[val_idx] = fi
else:
    print('StratifiedGroupKFold not available; using GroupKFold (CV may be less balanced).')
    gk = GroupKFold(n_splits=n_splits)
    for fi, (_, val_idx) in enumerate(gk.split(train_df, y, groups=groups)):
        folds[val_idx] = fi

assert (folds >= 0).all(), 'Fold assignment failed'
train_df['fold'] = folds

# Create label maps (category_id -> idx 0..K-1)
classes = np.sort(train_df[target_col].unique())
class_to_idx = {int(c): i for i, c in enumerate(classes)}
idx_to_class = {i: int(c) for i, c in enumerate(classes)}
train_df['target_idx'] = train_df[target_col].map(class_to_idx).astype(int)

# Class weights: (freq + 1e-3)^-0.5 normalized to mean=1
counts = train_df[target_col].value_counts().reindex(classes, fill_value=0).astype(float).values
weights = (counts + 1e-3) ** -0.5
weights = weights / weights.mean()
class_weights = {int(c): float(w) for c, w in zip(classes, weights)}

# Persist artifacts
out_dir = Path('./artifacts')
out_dir.mkdir(parents=True, exist_ok=True)
(DATA_DIR/'folds.csv').write_text(train_df[['id','file_name',target_col, group_col, 'fold']].to_csv(index=False))
json.dump({'class_to_idx': class_to_idx, 'idx_to_class': idx_to_class}, open(out_dir/'label_maps.json','w'))
json.dump({'class_weights': class_weights}, open(out_dir/'class_weights.json','w'))

# Save a compact meta for training
meta_cols = ['id','file_name', target_col, 'target_idx', 'fold', group_col, 'location', 'width', 'height', 'seq_num_frames', 'frame_num']
meta_cols = [c for c in meta_cols if c in train_df.columns]
train_df[meta_cols].to_csv(out_dir/'train_meta.csv', index=False)

# Print summary
print('Folds distribution:')
print(train_df.groupby('fold')[target_col].agg(['count']).reset_index())
print('Num classes:', len(classes))
print('Artifacts written to:', out_dir.resolve())

Folds distribution:
   fold  count
0     0  35901
1     1  35915
2     2  35826
3     3  35870
4     4  35910
Num classes: 14
Artifacts written to: /var/lib/simon/agent_run_states/iwildcam-2019-fgvc6-20250924-184738/artifacts


In [4]:
# Install CUDA 12.1 torch stack and core deps
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any preexisting torch stacks (best-effort)
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install EXACT cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze torch versions
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# 3) Install deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9', 'albumentations==1.4.8', 'opencv-python-headless',
    'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'seaborn',
    'einops', 'ema-pytorch',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity gate
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 575.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 437.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 416.0 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 233.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 449.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 577.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 479.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 439.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 467.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 480.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 518.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 188.4 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


In [6]:
# Smoke training: fold 0, tf_efficientnet_b3_ns @300 for 3 epochs with weighted CE + smoothing
import os, time, math, random, json
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import timm
from PIL import Image, ImageFile
import torchvision.transforms as T

torch.backends.cudnn.benchmark = True
ImageFile.LOAD_TRUNCATED_IMAGES = True

SEED = 42
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
seed_everything()

DATA_DIR = Path('.')
FAST_DIR = Path('./fast_storage/iwildcam')
IMG_DIR = FAST_DIR/'train_images'
assert IMG_DIR.exists(), 'Train images not found; ensure extraction complete.'

# Load metadata and build maps
df = pd.read_csv(DATA_DIR/'train.csv')
assert 'fold' in pd.read_csv(DATA_DIR/'folds.csv').columns, 'folds.csv missing or malformed'
folds_df = pd.read_csv(DATA_DIR/'folds.csv')
df = df.merge(folds_df[['id','fold']], on='id', how='left')
assert df['fold'].notna().all(), 'Some rows missing fold assignment'
classes = np.sort(df['category_id'].unique())
class_to_idx = {int(c): i for i, c in enumerate(classes)}
idx_to_class = {i: int(c) for i, c in enumerate(classes)}
df['target_idx'] = df['category_id'].map(class_to_idx).astype(int)
K = len(classes)

# Class weights (inverse sqrt freq, mean=1)
counts = df['category_id'].value_counts().reindex(classes, fill_value=0).astype(float).values
weights = (counts + 1e-3) ** -0.5
weights = weights / weights.mean()
class_weights_tensor = torch.tensor(weights, dtype=torch.float32).cuda()

IMG_SIZE = 300
VAL_SHORT = 336

train_tfm = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.6,1.0), interpolation=T.InterpolationMode.BICUBIC),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(num_magnitude_bins=31),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    T.RandomErasing(p=0.25, scale=(0.02,0.15), ratio=(0.3,3.3)),
])
val_tfm = T.Compose([
    T.Resize(VAL_SHORT, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class IWildCamDataset(Dataset):
    def __init__(self, df, img_dir, tfm, with_target=True):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.tfm = tfm
        self.with_target = with_target
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        fn = str(r['file_name'])
        p = self.img_dir/fn
        if not p.exists():
            p = self.img_dir/Path(fn).name
        try:
            img = Image.open(p).convert('RGB')
        except Exception:
            # fallback: create a black image to avoid crash but log index
            img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (0,0,0))
        img = self.tfm(img)
        if self.with_target:
            return img, int(r['target_idx']), r['seq_id'], r['id']
        else:
            return img, r['seq_id'], r['id']

def build_loaders(fold, bs=32, nw=8):
    tr = df[df['fold'] != fold].copy()
    va = df[df['fold'] == fold].copy()
    # Print val class distribution for sanity
    print('Fold', fold, 'val class distribution (category_id top-10):')
    print(va['category_id'].value_counts().head(10))
    ds_tr = IWildCamDataset(tr, IMG_DIR, train_tfm, with_target=True)
    ds_va = IWildCamDataset(va, IMG_DIR, val_tfm, with_target=True)
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True, num_workers=nw, pin_memory=True, persistent_workers=True, drop_last=True)
    dl_va = DataLoader(ds_va, batch_size=bs*2, shuffle=False, num_workers=nw, pin_memory=True, persistent_workers=True)
    return dl_tr, dl_va, va.reset_index(drop=True)

def macro_f1_from_logits_np(logits_np, targets_np, K):
    preds = logits_np.argmax(1)
    return f1_score(targets_np, preds, average='macro', labels=list(range(K)), zero_division=0)

def evaluate(model, dl, df_va, device, tta=False):
    model.eval()
    all_logits = []
    all_targets = []
    all_seq = []
    with torch.no_grad():
        for xb, yb, seqs, _ids in dl:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            logits = model(xb)
            if tta:
                xb_flip = torch.flip(xb, dims=[3])
                logits_flip = model(xb_flip)
                logits = 0.5*(logits + logits_flip)
            all_logits.append(logits.float().cpu())
            all_targets.append(yb.cpu())
            all_seq.extend(list(seqs))
    logits = torch.cat(all_logits, 0).numpy()
    targets = torch.cat(all_targets, 0).numpy()
    # Plain macro-F1 over all K classes
    f1_plain = macro_f1_from_logits_np(logits, targets, K)
    # Sequence-level averaging
    df_tmp = pd.DataFrame({'seq_id': all_seq})
    seq_indices = {}
    for i, s in enumerate(df_tmp['seq_id']):
        seq_indices.setdefault(s, []).append(i)
    logits_seq = logits.copy()
    for s, idxs in seq_indices.items():
        m = logits[idxs].mean(axis=0, keepdims=True)
        for i in idxs:
            logits_seq[i] = m
    f1_seq = macro_f1_from_logits_np(logits_seq, targets, K)
    return float(f1_plain), float(f1_seq)

def train_fold(fold=0, epochs=3, img_size=IMG_SIZE, bs=32, lr=3e-4, wd=1e-5):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Training fold {fold} for {epochs} epochs on {device}', flush=True)
    dl_tr, dl_va, df_va = build_loaders(fold, bs=bs, nw=8)
    model = timm.create_model('tf_efficientnet_b3_ns', pretrained=True, num_classes=K)
    model = model.to(device).to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    num_steps = epochs * len(dl_tr)
    warmup_steps = max(1, len(dl_tr))
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step+1)/float(warmup_steps)
        progress = (step - warmup_steps) / max(1, num_steps - warmup_steps)
        return 0.5*(1.0 + math.cos(math.pi * progress))
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda=lr_lambda)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.05)
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    best_f1 = -1.0
    ckpt_dir = Path('checkpoints'); ckpt_dir.mkdir(exist_ok=True, parents=True)
    t_start = time.time()
    global_step = 0
    for epoch in range(1, epochs+1):
        model.train()
        epoch_loss = 0.0
        t0 = time.time()
        for it, (xb, yb, _seqs, _ids) in enumerate(dl_tr):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=True):
                logits = model(xb)
                loss = criterion(logits, yb)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(opt)
            scaler.update()
            sch.step()
            epoch_loss += loss.item()
            global_step += 1
            if (it+1) % 100 == 0:
                elapsed = time.time()-t0
                print(f'Epoch {epoch} iter {it+1}/{len(dl_tr)} loss {epoch_loss/(it+1):.4f} elapsed {elapsed:.1f}s', flush=True)
        # Eval
        f1_plain, f1_seq = evaluate(model, dl_va, df_va, device, tta=True)
        print(f'Epoch {epoch} done. TrainLoss {epoch_loss/len(dl_tr):.4f} | Val F1 {f1_plain:.4f} | Val F1 seq-avg {f1_seq:.4f} | elapsed {time.time()-t0:.1f}s', flush=True)
        if f1_seq > best_f1:
            best_f1 = f1_seq
            ckpt_path = ckpt_dir/f'b3_fold{fold}_best.pth'
            torch.save({'model': model.state_dict(), 'f1_seq': best_f1, 'epoch': epoch}, ckpt_path)
            print('Saved best checkpoint:', ckpt_path, flush=True)
    print(f'Fold {fold} training complete. Best Val F1 seq-avg: {best_f1:.4f}. Total time {time.time()-t_start:.1f}s', flush=True)

# Run smoke training on fold 0
train_fold(fold=0, epochs=3, bs=32, lr=3e-4, wd=1e-5)

Training fold 0 for 3 epochs on cuda


Fold 0 val class distribution (category_id top-10):
category_id
0     25727
19     2137
1      1226
8      1208
11     1175
13     1047
16      986
17      810
3       609
18      357
Name: count, dtype: int64


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_235/3706356251.py:158: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/tmp/ipykernel_235/3706356251.py:171: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
/app/.pip-target/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


...

Epoch 2 iter 3400/4506 loss 1.4439 elapsed 383.7s


Epoch 2 iter 3500/4506 loss 1.4440 elapsed 395.0s


Epoch 2 iter 3600/4506 loss 1.4429 elapsed 406.3s


Epoch 2 iter 3700/4506 loss 1.4430 elapsed 417.6s


KeyboardInterrupt: 

In [ ]:
# Inference: TTA + optional sequence averaging, generate submission.csv aligned to sample_submission Ids
import os, json, math, time
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
from PIL import Image
import torchvision.transforms as T

DATA_DIR = Path('.')
FAST_DIR = Path('./fast_storage/iwildcam')
TEST_IMG_DIR = FAST_DIR/'test_images'
assert TEST_IMG_DIR.exists(), 'Test images dir missing'

# Load label maps from train
train_df_map = pd.read_csv(DATA_DIR/'train.csv')
classes = np.sort(train_df_map['category_id'].unique())
class_to_idx = {int(c): i for i, c in enumerate(classes)}
idx_to_class = {i: int(c) for i, c in enumerate(classes)}
K = len(classes)

# Load test and sample_submission to define required Id order
test_df = pd.read_csv(DATA_DIR/'test.csv')
has_seq = 'seq_id' in test_df.columns
n_test = len(test_df)
print('test.csv rows:', n_test, 'unique ids:', test_df['id'].nunique())
sample_df = pd.read_csv(DATA_DIR/'sample_submission.csv')
if sample_df.columns[0].startswith('Unnamed'):
    sample_df = sample_df.drop(columns=[sample_df.columns[0]])
assert set(['Id','Category']).issubset(sample_df.columns), 'sample_submission missing required columns'
sample_ids = sample_df['Id'].astype(str).values
print('sample_submission rows:', len(sample_ids))

IMG_SIZE = 300
VAL_SHORT = 336
val_tfm = T.Compose([
    T.Resize(VAL_SHORT, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class TestDataset(Dataset):
    def __init__(self, df, img_dir, tfm):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.tfm = tfm
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        fn = str(r['file_name']) if 'file_name' in r else str(r['id'])
        p = self.img_dir/fn
        if not p.exists():
            p = self.img_dir/Path(fn).name
        try:
            img = Image.open(p).convert('RGB')
        except Exception:
            img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (0,0,0))
        img = self.tfm(img)
        sid = r['seq_id'] if 'seq_id' in r else None
        iid = str(r['id']) if 'id' in r else Path(fn).stem
        return img, sid, iid

def infer_checkpoint(ckpt_path, bs=64, tta=True):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    ds = TestDataset(test_df, TEST_IMG_DIR, val_tfm)
    dl = DataLoader(ds, batch_size=bs, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)
    model = timm.create_model('tf_efficientnet_b3_ns', pretrained=False, num_classes=K)
    state = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(state['model'] if isinstance(state, dict) and 'model' in state else state, strict=False)
    model = model.to(device).to(memory_format=torch.channels_last)
    model.eval()
    all_logits = []
    all_seqs = []
    all_ids = []
    with torch.no_grad():
        for xb, seqs, ids in dl:
            xb = xb.to(device, non_blocking=True)
            logits = model(xb)
            if tta:
                xb_flip = torch.flip(xb, dims=[3])
                logits_flip = model(xb_flip)
                logits = 0.5*(logits + logits_flip)
            all_logits.append(logits.float().cpu())
            all_seqs.extend(list(seqs))
            all_ids.extend(list(ids))
    logits = torch.cat(all_logits, 0).numpy()
    return logits, np.array(all_seqs, dtype=object), np.array(all_ids, dtype=object)

# Choose checkpoint
ckpt1 = Path('checkpoints/b3_fold0_best.pth')
if not ckpt1.exists():
    alt = FAST_DIR/'b3_fold0_best.pth'
    ckpt1 = alt if alt.exists() else ckpt1
print('Using checkpoint:', ckpt1)

logits, seqs, ids = infer_checkpoint(ckpt1, bs=64, tta=True)
assert logits.shape[0] == len(test_df), f'logits rows {logits.shape[0]} != test rows {len(test_df)}'

# Sequence-level averaging on test if seq_id present
if has_seq:
    df_tmp = pd.DataFrame({'seq_id': seqs})
    seq_indices = {}
    for i, s in enumerate(df_tmp['seq_id']):
        seq_indices.setdefault(s, []).append(i)
    logits_seq = logits.copy()
    for s, idxs in seq_indices.items():
        m = logits[idxs].mean(axis=0, keepdims=True)
        for i in idxs:
            logits_seq[i] = m
    logits = logits_seq
else:
    print('seq_id not in test.csv; skipping sequence averaging')

# Aggregate duplicate ids by mean logits
df_pred = pd.DataFrame({'id': ids})
for k in range(K):
    df_pred[f'l{k}'] = logits[:, k]
agg = df_pred.groupby('id', as_index=False).mean()
print('Unique ids predicted:', len(agg))

# Map predictions to sample_submission Id order; fallback to class 0 for missing ids
logit_cols = [c for c in agg.columns if c.startswith('l')]
id_to_vec = {iid: agg.loc[i, logit_cols].values for i, iid in enumerate(agg['id'].astype(str).values)}
vec0 = np.zeros(K, dtype=np.float32); vec0[class_to_idx.get(0, 0)] = 1.0
pred_idx_list = []
miss = 0
for iid in sample_ids:
    v = id_to_vec.get(str(iid))
    if v is None:
        miss += 1
        v = vec0
    pred_idx_list.append(int(np.argmax(v)))
print('Missing ids in predictions (filled with 0):', miss)
pred_cat = [idx_to_class[i] for i in pred_idx_list]

sub = pd.DataFrame({'Id': sample_ids, 'Category': pred_cat})
assert list(sub.columns) == ['Id','Category'] and len(sub) == len(sample_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape, 'and columns', list(sub.columns))

test.csv rows: 16877 unique ids: 16862
sample_submission rows: 16877
Using checkpoint: checkpoints/b3_fold0_best.pth


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_235/978938898.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pl